In [1]:
!pip install qiskit
!pip install pylatexenc

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.8/943.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.3 MB/s eta 0:00:00
  Created wheel for qiskit: filename=qiskit-0.39.4-py3-none-any.whl size=12274 sha256=4c5143c8b587710d94da4d2a480d8688674ff2d4917b1f6cb97f618655173bc1
  Stored in directory: /root/.cache/pip/wheels/de/e2/bc/795993e2fc0f60127e9f418d52ba163246c910e81a53e859f4
Successfully built qiskit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 344.2 kB/s eta 0:00:00
  Preparing metadata (setup.p

In [2]:
#import libraries

import random
import numpy as np
from math import *

from qiskit import *
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

In [3]:
#obtain random binary list

def random_list_function(num):
    
    random_list = []

    for iterator in range(num):
        bit = random.randint(0, 1)
        random_list.append(bit)
        
    return random_list

In [4]:
#prepare the random hadamard state, according to a given random list

def random_hadamard_state(circuit, main_indices, random_list):
    
    for iterator in main_indices:
        if random_list[iterator] == 1:
            circuit.x(iterator)

    circuit.barrier()
    
    for iterator in main_indices:
        circuit.h(iterator)
    
    return circuit

In [5]:
#make a round robin of an list in order to build our simplicial projector

def round_robin(list):
    
    #create the rounds
    rounds = [] 
    
    #judge the parity
    if len(list) % 2 == 1:
        list = list + ["NULL"]
       
    #fill elements in our rounds
    for iterator in range(len(list)-1):
        
        #create one element
        round = [] 
        
        #divide the list to 2 groups
        mid = int(len(list) / 2)
        first_line = list[:mid]
        second_line = list[mid:]
        second_line.reverse()
        
        #select competitors
        for iter in range(mid):
            
            #skip the competitor on rest
            if first_line[iter] == "NULL" or second_line[iter] == "NULL":
                continue

            #match competitors
            match = []
            match.append(first_line[iter])
            match.append(second_line[iter])
            
            #finish one round
            round.append(match)
        
        #add this round to rounds
        rounds.append(round)
        
        #change the order of list to continue
        list.insert(1, list.pop())
    
    return rounds

In [6]:
#build a simplicial projector

def simplicial_projector(circuit,
                        simplices,
                        main_indices,
                        ancilla_indices,
                        ancilla_meas_cbits):
    
    vertices = [simplex[0] for simplex in simplices if len(simplex) == 1]
    edges = [set(simplex) for simplex in simplices if len(simplex) == 2]
    
    num = len(main_indices)
    assert(len(vertices) == num)
    
    #round robin
    rounds = round_robin([iter for iter in range(num)])
    
    #reset ancillas
    for a_i in ancilla_indices:
        circuit.reset(a_i)
        
    #for each round,
    for rou_iter,round in enumerate(rounds):
        #for each match in that round,
        for mat_iter, match in enumerate(round):
            #if the edge is not in the simplicial complex
            if set(match) not in edges:
                # apply a toffoli and measurement
                circuit.ccx(main_indices[match[0]],
                            main_indices[match[1]],
                            ancilla_indices[mat_iter])
                circuit.measure(ancilla_indices[mat_iter],
                               ancilla_meas_cbits[mat_iter + floor(num / 2) * rou_iter])
    
    return circuit

In [7]:
#make an increment projector in order to build our count projector

def increment_operator(circuit, main_list, borrow_list):
    
    #examine whether the number of borrow quibit =  the number of main quibit - 2
    assert(len(borrow_list) == len(main_list) - 2)
    
    #construct the circuit
    for iter in range(len(main_list) - 2):
        if iter == 0:
            circuit.ccx(main_list[0], main_list[iter + 1], borrow_list[iter])
        else:
            circuit.ccx(borrow_list[iter - 1], main_list[iter + 1], borrow_list[iter])
    
    circuit.cx(borrow_list[len(borrow_list) - 1], main_list[len(main_list) - 1])

    for iter in reversed(range(len(main_list) - 2)):
        if iter == 0:
            circuit.ccx(main_list[0], main_list[iter + 1], borrow_list[iter])
            circuit.cx(main_list[0], main_list[iter + 1])
        else:
            circuit.ccx(borrow_list[iter - 1], main_list[iter + 1], borrow_list[iter])
            circuit.cx(borrow_list[iter - 1], main_list[iter + 1])
    
    circuit.x(main_list[0])
    
    return circuit

In [8]:
#build a count projector

def count_projector(circuit, main_indices, counter_indices, counter_ancilla_indices, counter_cbits):
    
    #give the set of main qubits
    for iterator in range(len(main_indices)):
        #let a member of main qubits together with counter indices
        main_list = [main_indices[iterator]] + counter_indices
        #Start to use counter ancilla qubits to help constructing a increment gate.
        borrow_list = counter_ancilla_indices[:(len(counter_ancilla_indices) - 1)]
        #create the increment gate
        increment_operator(circuit, main_list, borrow_list)
        #add X-gate to this main qubit, to make it be a control-qubit
        circuit.x(main_indices[iterator])
        
    #make a measure
    circuit.measure(counter_indices, counter_cbits)
    
    return circuit

In [9]:
#build a evolve boundary projector
#all according to "Basic circuit to simulate Boundary Matrix"

def evolve_boundary_operator(circuit, main_indices, main_control_ancilla_index, t):
    
    for row in range(len(main_indices)):
        
        for column in range(row):
            circuit.cx(column,main_control_ancilla_index)
    
        circuit.h(row)
        circuit.cx(row, main_control_ancilla_index)
        circuit.rz(t, main_control_ancilla_index)
        circuit.cx(row, main_control_ancilla_index)
        circuit.h(row)
    
        for column in reversed(range(row)):
            circuit.cx(column, main_control_ancilla_index)
    
    return circuit

In [10]:
#build the whole build moment estimation circuit

def build_moment_estimation_circuit(simplices,
                                    iterations,
                                    random_list,
                                    t = 0.1,
                                    seed = None):
    
    ##########
    # set up #
    ##########
    
    #determine number of main qubits
    vertices = [simplex[0] for simplex in simplices if len(simplex) == 1]
    num = len(vertices)
    
    #calculate the number of ancilla qubits needed
    aq1 = floor(num / 2)
    aq2 = 1 + 2 * ceil(log2(num+1))
    num_ancilla_qubits = max(aq1, aq2)
    
    #calculate the number of classical bits needed
    num_cbits = ceil(log2(num+1)) * (iterations // 2 + 1) + (iterations + 1) * (num * (num-1)) // 2
    
    circuit = QuantumCircuit(QuantumRegister(num), AncillaRegister(num_ancilla_qubits), ClassicalRegister(num_cbits))
    
    main_indices = [iter for iter in range(num)]
    
    #always use these indices of acillas for count projectors
    main_control_ancilla_index = num
    counter_indices = [iter for iter in range(num + 1, num + 1 + ceil(log2(num + 1)))]
    counter_ancilla_indices = [iter for iter in range(num + 1 + ceil(log2(num + 1)), num + 1 + ceil(log2(num + 1)) * 2)]
    
    #always use these indices of acillas for simplicial projectors
    ancilla_indices = [iter for iter in range(num, num + floor(num / 2))]
    
    #offset for simplicial projector cbits
    #add this to indices to get to the cbits for the simplicial projector
    sp_offset = ceil(log2(num+1)) * (iterations // 2 + 1)
    
    
    #################
    # build circuit #
    #################
    
    i = 0
    
    # hadamard state
    circuit = random_hadamard_state(circuit, main_indices, random_list)
    
    circuit.barrier()
    
    # count projector
    counter_cbits = [iter for iter in range((i // 2) * ceil(log2(num + 1)), (i // 2 + 1) * ceil(log2(num + 1)))]
    circuit = count_projector(circuit, 
                              main_indices, 
                              counter_indices, 
                              counter_ancilla_indices, 
                              counter_cbits)
    
    circuit.barrier()

    # simplicial projector
    ancilla_meas_cbits = [iter for iter in range(sp_offset + i * (num * (num - 1)) // 2, sp_offset + (i + 1) * (num * (num - 1)) // 2)]
    circuit = simplicial_projector(circuit,
                                   simplices,
                                   main_indices,
                                   ancilla_indices,
                                   ancilla_meas_cbits)
    
    circuit.barrier()
    
    for i in range(1, iterations + 1):
        circuit = evolve_boundary_operator(circuit, main_indices, main_control_ancilla_index, t)
        
        circuit.barrier()
        
        ancilla_meas_cbits = [iter for iter in range(sp_offset + i * (num * (num - 1)) // 2, sp_offset + (i + 1) * (num * (num - 1)) // 2)]
        circuit = simplicial_projector(circuit,
                                       simplices,
                                       main_indices,
                                       ancilla_indices,
                                       ancilla_meas_cbits)
        
        circuit.barrier()
        
        if (i % 2 == 0):
            counter_cbits = [iter for iter in range((i // 2) * ceil(log2(num + 1)), (i // 2 + 1) * ceil(log2(num + 1)))]
            circuit = count_projector(circuit, 
                                      main_indices, 
                                      counter_indices, 
                                      counter_ancilla_indices, 
                                      counter_cbits)
            circuit.barrier()
                                      
    
    return circuit

In [11]:
#string matching functions to backup

#get the valid value of the result from count measurements in our circuit
def retur_rep_k(k_value, vertice_num, iterations):
    
    assert(k_value <= vertice_num)
    
    i = 0
    
    #get the ([log2(n + 1)] + 1)-bit binary string of a given integer k (counted_bits),
    #e.g. k = 4, n = 5, counted_bits = '0100'
    counted_bits = bin(k_value)[2:].zfill(ceil(log2(vertice_num + 1)))
    
    #repeat the counted_bits according to times the count_measurements appear in our circuit
    #e.g. 3 times, counted_bits = '0100', rep_bits = '010001000100'
    rep_bits = '' + counted_bits
    for i in range(1, iterations + 1):
        if (i % 2 == 0):
            rep_bits += counted_bits
    
    return rep_bits

#get the valid value of the result from all measurements in our circuit
def validation(txt, rep_bits):
    
    #get the txt_tail to verify whether it's equal to rep_bits
    txt_tail = txt[len(txt)-len(rep_bits):len(txt)]
    
    if txt_tail == rep_bits:
        #get the txt_head to verify whether it's ALL equal to '0'
        txt_head = txt[0:len(txt)-len(rep_bits)]
        for iter in txt_head:
            if iter == '0':
                continue
            return False
        return True
    else:
        return False

#combine two above
def comparation(txt, k_value, vertice_num, iterations):
    
    rep_bits = retur_rep_k(k_value, vertice_num, iterations)
    
    return validation(txt,rep_bits)

In [12]:
#simulate our circuit by 'QasmSimulator'
#But if we have a feasible real hardware, this block of function can be replaced.
def circuit_simulation(circuit, shots = 1000):
    #Create a simulator
    simulator = QasmSimulator()
    #Compile the simulator
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots = shots)

    #Count the probably states
    result = job.result()
    counts = result.get_counts(compiled_circuit)

    return counts

In [13]:
#get the value of mu in one iteration
def get_mu_number(counts, k_value, vertice_num, iterations):
    
    #get all of our results, then select ones have valid states values
    list = [value for key, value in counts.items() if comparation(key, k_value, vertice_num, iterations)]
    
    #estimate the probabilities of psi, which is mu
    integer = 0
    for iter in range(len(list)):
        integer += list[iter]
    mu = integer / counts.shots()
    
    return mu

In [14]:
#math functions to backup

#combination number of n & k
#Only python 3.6 is available in kaggle,
#if you have more advanced version, this function can be replaced.
def CombinationNum(n,k):
    
    if n < k:
        raise TypeError("Wrong Combination Number") 
    
    numerator = 1
    denominator = 1
    
    for itera in range(k+1, n+1):  
        numerator *= itera
        itera += 1
        
    for itera in range(1, n-k+1):  
        denominator *= itera
        itera += 1
        
    result = int(numerator / denominator)
    
    return result

#g(j, i)
def gFunction(j, i):
    number = CombinationNum(2 * i, i) * CombinationNum(j, 2 * i) / CombinationNum(j - 1, i)
    return number

'''
#polynomial of T(x), non-linear version  We won't used it in this process.

def TFunction(j, x):
    
    sum = 0
    
    num = int(j/2)
    
    for i in range(num):
        sum += ((-1) ** i) * (2 ** (j - (2*i + 1))) * gFunction(j, i) * (x ** (j - 2*i))
        
    return int(sum)
'''
#polynomial of T(x), linear version, 'cause the series of mu have been derived, which is obtain from Lap^i.
def Tfunction_of_Lap(j, mus):
    
    sum = 0
    
    num = floor(j/2)
    
    for i in range(num):
        sum += ((-1) ** i) * (2 ** (j - (2*i + 1))) * gFunction(j, i) * mus[j - 2*i]
        
    return sum

#coefficients aka 'c_j', but I don't know how to set a & b.
def Coef(m, a, b):
    
    coef_list = []
    
    coef = np.reciprocal(np.pi) * (np.reciprocal(np.cos(a)) - np.reciprocal(np.cos(b)))
    coef_list.append(coef)
    
    for j in range(1, m + 1):
        coef = (2 * (np.sin(j * np.reciprocal(np.cos(a))) - np.sin(j * np.reciprocal(np.cos(b)))))/(np.pi * j)
        coef_list.append(coef)
        
    return coef_list

In [15]:
#inner function of rank estimation process
def Function_inner(m, simplices, k_value, shots):
    
    vertices = [simplex[0] for simplex in simplices if len(simplex) == 1]
    num = len(vertices)
    
    #get the random list of this stochastic step
    random_list = random_list_function(num)
    
    mus = []
    for iter in range(0, m+1):
        #build moment estimation circuit of this iteration
        circuit = build_moment_estimation_circuit(simplices, iter, random_list)
        #simulate it
        counts = circuit_simulation(circuit, shots = shots)
        #get mu of this iteration
        mu = get_mu_number(counts, k_value, num, iter)
        #record
        mus.append(mu)
        
    theta_l = []
    for iter in range(0, m+1):
        #calculate a theta according to the list of mu
        theta_l_j = Tfunction_of_Lap(iter, mus)
        #record
        theta_l.append(theta_l_j)
        
    return theta_l

In [16]:
#rank estimation
def derive_rank_estimation(n_v, m, simplices, k_value, shots):

    element = []
    
    #stochastic steps
    for l in range(n_v):
        #derive the set of thetas in this step
        theta_l = Function_inner(m, simplices, k_value, shots)
        #coefficients [I don't know how to set a & b.]
        coef_list = Coef(m, 0.5, 1.5)
        
        #derive the element in this step
        assert(len(theta_l) == len(coef_list))
        element_l = sum(np.multiply(theta_l, coef_list))
        element.append(element_l)
    
    #accomplish estimating our rank!
    rank_estimation = round(sum(element) / n_v)

    return rank_estimation

In [17]:
#just an example

simplices = [[0],[1],[2],[3],[0,1],[3,2],[1,2],[1,3],[2,4],[3,4],[1,2,3],[2,3,4]]

rank_estimation = derive_rank_estimation(4, 3, simplices, 1, 1000)

print(rank_estimation)

0
